In [2]:
import numpy as np
import trimesh
import matplotlib.pyplot as plt
import copy

In [34]:
face = trimesh.load("Face.obj")

In [35]:
def boundary(mesh):
    """ A function to find the boundary edges and vertices of a mesh. """

    next_vertex = {}
    boundary_edges = []
    boundary_vertices = []

    edges = mesh.edges
    vertex_faces = mesh.vertex_faces

    for i in range(len(edges)):

        edge = edges[i]
        v1, v2 = edge
        v1_faces = [j for j in vertex_faces[v1] if j != -1]
        v1v2_faces = [j for j in v1_faces if j in vertex_faces[v2]]
        
        if len(v1v2_faces) == 1:
            
            next_vertex[v1] = v2
            boundary_edges  = boundary_edges + [edge]
            boundary_vertices = boundary_vertices + [v1, v2]
        
    boundary_vertices = np.unique(np.array(boundary_vertices))
    
    return boundary_edges, boundary_vertices, next_vertex

In [233]:
face = trimesh.load("lilium.obj")
vertices = face.vertices
new_vertices = np.copy(vertices)
bbox_min, bbox_max = face.bounds

# get corners of mesh in x,y
corner_vertices = [
    [bbox_min[0], bbox_min[1]],
    [bbox_max[0], bbox_min[1]],
    [bbox_max[0], bbox_max[1]],
    [bbox_min[0], bbox_max[1]]
]

corner_vertex_indices = []
# find corners
for corner in corner_vertices:
    # calculate distance (in terms of x and y)
    dist = np.linalg.norm(vertices[:, :2] - np.array(corner), axis=1)
    
    # find the index of the closest vertex
    closest_idx = np.argmin(dist)
    # keep track for later
    corner_vertex_indices.append(closest_idx)

    # map x,y to unit cube corners based on positive/negative values
    new_vertices[closest_idx][:2] = np.clip(vertices[closest_idx][:2] >= 0, 0.0, 1.0)
    # set to same value in z to flatten to 2D
    new_vertices[closest_idx][2] = -3.0 #set it back for clarity

# find boundaries
_, _, next_vertex = boundary(face)

# map to unit sphere
# starting index 
next_vertex_idx = next_vertex[corner_vertex_indices[0]]
# keep track of vertices to move
vertex_indices = [corner_vertex_indices[0]]
for i in range(len(next)):
    vertex_indices.append(next_vertex_idx)
    next_vertex_idx = next[next_vertex_idx]
    
    if(next_vertex_idx in corner_vertex_indices):
        vertex_indices.append(next_vertex_idx)
        vertices_sub = vertices[vertex_indices]
        A = vertices_sub[0] # get one corner
        B = vertices_sub[-1] # get second corner

        distances = []
        for j in range(len(vertices_sub)-1):
            current_vertex = vertices_sub[j]
            next_vertex = vertices_sub[j + 1]
            dist = np.linalg.norm(next_vertex - current_vertex)
            distances.append(dist)

        dist_sum = np.sum(distances)
        new_vertices[vertex_indices[1:-1]] = [dist/dist_sum * np.linalg.norm(B-A) + A  for dist in distances[:-1]]

        # WILL CHANGE THIS QUICK FIX FOR SETTING ALL y to same value
        mask = np.zeros(new_vertices.shape[0], dtype=bool)
        mask[vertex_indices] = True
        new_vertices[mask, 2] = -3.0

        vertex_indices = [next_vertex_idx]
        next_vertex_idx = next[next_vertex_idx]

# Save new mesh
new_mesh = copy.deepcopy(face)
new_mesh.vertices = new_vertices
new_mesh.export("lillium_to_square.obj")

'# https://github.com/mikedh/trimesh\nv -0.55953800 0.51809000 0.06617230\nv -0.54615500 0.47721100 0.07876800\nv -0.53907900 0.53867900 0.04229500\nv -0.44586200 0.42009500 0.05852490\nv -0.61205000 0.39787500 0.10618400\nv -0.75680500 0.43238400 0.14320100\nv -0.76177800 0.35572100 0.13511800\nv -0.67578500 0.55976600 0.09665780\nv -0.57546800 0.21404700 0.01806990\nv -0.61430500 0.21660700 0.03870620\nv -0.73505000 0.22282700 0.09569290\nv -0.76747800 0.16210700 0.08698550\nv -0.80924200 0.25123600 0.11654000\nv -0.85515300 0.25064200 0.11422300\nv -0.81255300 0.11897600 0.07754770\nv -0.93796200 0.27571600 0.09855710\nv -0.93195300 0.23552500 0.08524840\nv -0.76992400 0.31901600 0.12912900\nv -0.94346000 0.07032590 -0.00155173\nv -0.78221200 0.09223790 0.07095030\nv 0.09513000 0.13303600 -0.01623220\nv 0.13487500 0.13989800 -0.01952270\nv 0.15232400 0.10824600 -0.03431440\nv 0.09110800 0.06496640 -0.03333380\nv -0.01230950 0.11979600 -0.02915850\nv 0.04562560 0.07001510 -0.03163940